In [9]:
import matplotlib.pyplot as plt
import json
from PIL import (
    Image,
    ImageFilter,
    ImageEnhance,
)
from sklearn import svm
from itertools import product
from functools import reduce
import skimage
from skimage.feature import (
    hog,
    daisy,
    blob_dog,
)
import numpy
from numpy import array
import traceback
from operator import mul

In [10]:
def preprocess_color(image):
    image = image
    return image

def preprocess_grayscale(image):
    image = image.convert("L")
    return image

def preprocess_contour(image):
    image = image.filter(ImageFilter.CONTOUR)
    return image

def preprocess_grayscale_contour(image):
    image = image.convert("L").filter(ImageFilter.CONTOUR)
    return image

def preprocess_sharpen(image):
    image = image.filter(ImageFilter.SHARPEN)
    return image

def preprocess_grayscale_sharpen(image):
    image = image.convert("L").filter(ImageFilter.SHARPEN)
    return image

def preprocess_blur(image):
    image = image.filter(ImageFilter.BLUR)
    return image

def preprocess_grayscale_blur(image):
    image = image.convert("L").filter(ImageFilter.BLUR)
    return image

def preprocess_find_edges(image):
    image = image.filter(ImageFilter.FIND_EDGES)
    return image

def preprocess_grayscale_find_edges(image):
    image = image.convert("L").filter(ImageFilter.FIND_EDGES)
    return image

def preprocess_combined_filter(image):
    image = image.filter(ImageFilter.BLUR).filter(ImageFilter.FIND_EDGES)
    return image

def preprocess_grayscale_combined_filter(image):
    image = image.convert("L").filter(ImageFilter.BLUR).filter(ImageFilter.FIND_EDGES)
    return image

In [11]:
def extract_base_image(image, visualize=False):
    if visualize:
        return (numpy.asarray(image), numpy.asarray(image))
    else:
        return numpy.asarray(image)


def extract_hog(image, visualize=False):
    return hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=visualize)

def extract_sift(image, visualize=False):
    return daisy(image, visualize=visualize)

def extract_blob_dog(image, visualize=False):
    return blob_dog(image, visualize=visualize)

In [12]:
color_preprocessing_techniques = [
    preprocess_color,
    preprocess_contour,
    preprocess_sharpen,
    preprocess_blur,
    preprocess_find_edges,
    preprocess_combined_filter,
]
grayscale_preprocessing_techniques = [
    preprocess_grayscale, 
    preprocess_grayscale_contour,
    preprocess_grayscale_sharpen,
    preprocess_grayscale_blur,
    preprocess_grayscale_find_edges,
    preprocess_grayscale_combined_filter,
]
extraction_techniques = [
    extract_hog,
    extract_base_image,
]
grayscale_extraction_techniques = [
    extract_sift,

]

In [13]:
def get_model_accuracy(classifier, X, Y):
    classifications = classifier.predict(X)
    successful = 0
    for (tX, tY) in list(
        zip(Y, classifications)
    ):
        if tX == tY:
            successful += 1
    return successful/len(Y)

In [14]:
def run_pipeline(classifier, preprocess, extract):
    scores = {}
    with open("data/data_set/testing/config_easy.json") as json_file:
        testing_data = json.loads(json_file.read())
        X = []
        Y = []
        for image_data in testing_data:
            image_path = "./data/data_set/testing/{}".format(image_data['path'])
            image = Image.open(image_path).resize((165,165))
            X.append(extract(preprocess(image)))
            Y.append(image_data["angle"])
        X = array(X)
        Y = array(Y)
        shape = X.shape
        X = X.reshape((shape[0],reduce(mul, shape[1:], 1)))
        scores["easy"] = get_model_accuracy(classifier, X, Y)
        
    with open("data/data_set/testing/config_hard.json") as json_file:
        testing_data = json.loads(json_file.read())
        X = []
        Y = []
        for image_data in testing_data:
            image_path = "./data/data_set/testing/{}".format(image_data['path'])
            image = Image.open(image_path).resize((165,165))
            X.append(extract(preprocess(image)))
            Y.append(image_data["angle"])
        X = array(X)
        Y = array(Y)
        shape = X.shape
        X = X.reshape((shape[0],reduce(mul, shape[1:], 1)))
        scores["hard"] = get_model_accuracy(classifier, X, Y)
        
    if "easy" in scores and "hard" in scores:
        scores["average"] = (scores["easy"] + scores["hard"]) / 2
    
    return scores

In [15]:
def create_classifier(preprocess, extract):
    with open("data/data_set/training/config.json") as json_file:
        training_data = json.loads(json_file.read())
        X = []
        Y = []
        for image_data in training_data:
            image_path = "./data/data_set/training/{}".format(image_data['path'])
            image = Image.open(image_path).resize((165,165))
            X.append(extract(preprocess(image)))
            Y.append(image_data["angle"])
        
        X = array(X)
        Y = array(Y)
        shape = X.shape
        X = X.reshape((shape[0],reduce(mul, shape[1:], 1)))
        clf = svm.SVC(gamma="scale", decision_function_shape='ovo')
        clf_fit = clf.fit(X, Y)
        return clf_fit


In [16]:
data = []

for preprocess, extract in product(color_preprocessing_techniques, extraction_techniques):
    print(preprocess, extract)
    classifier = create_classifier(preprocess, extract)
    scores = run_pipeline(classifier, preprocess, extract)
    data.append((scores, (preprocess, extract)))
    
for preprocess, extract in product(grayscale_preprocessing_techniques, extraction_techniques):
    print(preprocess, extract)
    classifier = create_classifier(preprocess, extract)
    scores = run_pipeline(classifier, preprocess, extract)
    data.append((scores, (preprocess, extract)))
    
for preprocess, extract in product(grayscale_preprocessing_techniques, grayscale_extraction_techniques):
    print(preprocess, extract)
    classifier = create_classifier(preprocess, extract)
    scores = run_pipeline(classifier, preprocess, extract)
    data.append((scores, (preprocess, extract)))

<function preprocess_color at 0x7efc843dfbf8> <function extract_hog at 0x7efc843dfc80>
<function preprocess_color at 0x7efc843dfbf8> <function extract_base_image at 0x7efc843dfb70>
<function preprocess_contour at 0x7efc843dfea0> <function extract_hog at 0x7efc843dfc80>
<function preprocess_contour at 0x7efc843dfea0> <function extract_base_image at 0x7efc843dfb70>
<function preprocess_sharpen at 0x7efc843df9d8> <function extract_hog at 0x7efc843dfc80>
<function preprocess_sharpen at 0x7efc843df9d8> <function extract_base_image at 0x7efc843dfb70>
<function preprocess_blur at 0x7efc843df8c8> <function extract_hog at 0x7efc843dfc80>
<function preprocess_blur at 0x7efc843df8c8> <function extract_base_image at 0x7efc843dfb70>
<function preprocess_find_edges at 0x7efc843df7b8> <function extract_hog at 0x7efc843dfc80>
<function preprocess_find_edges at 0x7efc843df7b8> <function extract_base_image at 0x7efc843dfb70>
<function preprocess_combined_filter at 0x7efcc0580268> <function extract_hog a

In [17]:
data

[({'easy': 0.782608695652174, 'hard': 0.6875, 'average': 0.7350543478260869},
  (<function __main__.preprocess_color(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.782608695652174,
   'hard': 0.7291666666666666,
   'average': 0.7558876811594203},
  (<function __main__.preprocess_color(image)>,
   <function __main__.extract_base_image(image, visualize=False)>)),
 ({'easy': 0.6521739130434783, 'hard': 0.75, 'average': 0.7010869565217391},
  (<function __main__.preprocess_contour(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.6956521739130435, 'hard': 0.5625, 'average': 0.6290760869565217},
  (<function __main__.preprocess_contour(image)>,
   <function __main__.extract_base_image(image, visualize=False)>)),
 ({'easy': 0.6956521739130435,
   'hard': 0.4791666666666667,
   'average': 0.5874094202898551},
  (<function __main__.preprocess_sharpen(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({

In [18]:
sorted(data, key=lambda data: data[0]["easy"],reverse=True)

[({'easy': 0.9565217391304348, 'hard': 0.8125, 'average': 0.8845108695652174},
  (<function __main__.preprocess_grayscale(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.9130434782608695,
   'hard': 0.8333333333333334,
   'average': 0.8731884057971014},
  (<function __main__.preprocess_blur(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.9130434782608695,
   'hard': 0.7708333333333334,
   'average': 0.8419384057971014},
  (<function __main__.preprocess_grayscale_contour(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.9130434782608695,
   'hard': 0.8541666666666666,
   'average': 0.8836050724637681},
  (<function __main__.preprocess_grayscale_blur(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.8695652173913043,
   'hard': 0.7291666666666666,
   'average': 0.7993659420289855},
  (<function __main__.preprocess_grayscale_sharpen(image)>,
   <funct

In [19]:
sorted(data, key=lambda data: data[0]["hard"],reverse=True)

[({'easy': 0.9130434782608695,
   'hard': 0.8541666666666666,
   'average': 0.8836050724637681},
  (<function __main__.preprocess_grayscale_blur(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.9130434782608695,
   'hard': 0.8333333333333334,
   'average': 0.8731884057971014},
  (<function __main__.preprocess_blur(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.9565217391304348, 'hard': 0.8125, 'average': 0.8845108695652174},
  (<function __main__.preprocess_grayscale(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.9130434782608695,
   'hard': 0.7708333333333334,
   'average': 0.8419384057971014},
  (<function __main__.preprocess_grayscale_contour(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.7391304347826086,
   'hard': 0.7708333333333334,
   'average': 0.754981884057971},
  (<function __main__.preprocess_grayscale_combined_filter(image)>,
  

In [20]:
sorted(data, key=lambda data: data[0]["average"],reverse=True)

[({'easy': 0.9565217391304348, 'hard': 0.8125, 'average': 0.8845108695652174},
  (<function __main__.preprocess_grayscale(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.9130434782608695,
   'hard': 0.8541666666666666,
   'average': 0.8836050724637681},
  (<function __main__.preprocess_grayscale_blur(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.9130434782608695,
   'hard': 0.8333333333333334,
   'average': 0.8731884057971014},
  (<function __main__.preprocess_blur(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.9130434782608695,
   'hard': 0.7708333333333334,
   'average': 0.8419384057971014},
  (<function __main__.preprocess_grayscale_contour(image)>,
   <function __main__.extract_hog(image, visualize=False)>)),
 ({'easy': 0.8695652173913043,
   'hard': 0.7291666666666666,
   'average': 0.7993659420289855},
  (<function __main__.preprocess_grayscale_sharpen(image)>,
   <funct

# Testing the pipelines on images from the virtual environment

In [21]:
image_dir = "/home/mega/Documents/scripts_n406_working/test_img_file/cv_images_3_06"
virtual_images = [Image.open("{}/{}".format(image_dir, image_name)).resize((165,165)) for image_name in [
    "0.png",
    "0deg.png",
    "45deg.png",
    "90deg.png",
    "135deg.png",
    "135deg_2.png",
    "not_classefiable.png"
]]
comparison_images = [Image.open("{}/{}".format(image_dir, image_name)).resize((165,165)) for image_name in [
    "img8_7.png",
    "img1_1.png",
    "135_crazyfly_up.png",
    "90deg_box4_up.png",
    "img8_4.png",
    "img5_4.png",
    "not_classefiable.png"
]]

In [22]:
def explore_pipeline(preprocess, extraction):
    for virtual, comparison in zip(virtual_images, comparison_images):
        processed_virtual = preprocess(virtual)
        processed_comparison = preprocess(comparison)
        _, virtual_features = extraction(processed_virtual, visualize=True)
        _, comparison_features = extraction(processed_comparison, visualize=True)
    
        processed_virtual = preprocess(virtual).convert("RGB")
        processed_comparison = preprocess(comparison).convert("RGB")
        extracted_virtual = Image.fromarray(virtual_features).resize((165,165)).convert("RGB")
        extracted_comparison = Image.fromarray(comparison_features).resize((165,165)).convert("RGB")
    
        side_initial = numpy.concatenate((numpy.array(virtual),numpy.array(comparison)), axis=1)
        side_processed = numpy.concatenate((numpy.array(processed_virtual),numpy.array(processed_comparison)), axis=1)
        side_extracted = numpy.concatenate((numpy.array(extracted_virtual),numpy.array(extracted_comparison)), axis=1)
        Image.fromarray(numpy.concatenate((side_initial,side_processed, side_extracted))).show()

In [31]:
explore_pipeline(preprocess_grayscale, extract_hog)

In [46]:
clf = create_classifier(preprocess_grayscale, extract_hog)

clf.predict([extract_hog(preprocess_grayscale(comparison_images[1]))])
# virtual_images[0].show()

array([90])